# PseudoArcLengthContinuation

## Loading packages and functions

In [1]:
using LinearAlgebra
using Plots
using NLsolve

LoadError: ArgumentError: Package NLsolve not found in current path.
- Run `import Pkg; Pkg.add("NLsolve")` to install the NLsolve package.

In [2]:
include("../src/PseudoArcLengthContinuation.jl")

PseudoArcLength (generic function with 6 methods)

## Problem

### F function

In [3]:
function F_Example1!(F,u,p)

   x=u[1]
   y=u[2]
   z=u[3]
   F[1]=x^2+y^2+z^2-6          #F[1]=x^2+y^2+z^2-4
   F[2]=x+y-z
   F[3]=(x-1)^2-1
    
   return nothing 
end

F_Example1! (generic function with 1 method)

In [4]:
function J_Example1!(J,u,p)

   x=u[1]
   y=u[2]
   z=u[3]
   J[1,1]=2*x
   J[1,2]=2*y
   J[1,3]=2*x
   J[2,1]=1
   J[2,2]=1
   J[2,3]=-1
   J[3,1]=(x-1)
   J[3,2]=0
   J[3,3]=0 
    
   return nothing 
end

J_Example1! (generic function with 1 method)

In [5]:
#sol = nlsolve(F_Example1!,J_Example1!, [2., -1.,  1.])
#sol.zero

### F* function

In [6]:
function F2_Example1!(F,u,p,βj,uj,Δtj)

   x=u[1]
   y=u[2]
   z=u[3]

   xj=uj[1]
   yj=uj[2]
   zj=uj[3]

   F_Example1!(F,u,p)
   
   # βj*(x-xj)-Δtj 
   F[3]=βj[1]*(x-xj)+βj[2]*(y-yj)+βj[3]*(z-zj)-Δtj
    
   return nothing 
end

"""
u=rand(3)
p=nothing
uj=rand(3)
βj=rand(3)
Δj=1.
F=zeros(6)
F2_Example1!(F,u,βj,uj,Δj)
"""

"u=rand(3)\np=nothing\nuj=rand(3)\nβj=rand(3)\nΔj=1.\nF=zeros(6)\nF2_Example1!(F,u,βj,uj,Δj)\n"

In [6]:
function J2_Example1!(J,u,p,βj,uj,Δtj)

   x=u[1]
   y=u[2]
   z=u[3]

   J_Example1!(J,u,p)
    
   J[3,1]=βj[1]
   J[3,2]=βj[2]
   J[3,3]=βj[3]
    
   return nothing 
end

"""
u=rand(3)
p=nothing
uj=rand(3)
βj=rand(3)
Δj=1.
J=zeros(6,3)
J2_Example1!(J,u,βj,uj,Δj)

"""

"u=rand(3)\np=nothing\nuj=rand(3)\nβj=rand(3)\nΔj=1.\nJ=zeros(6,3)\nJ2_Example1!(J,u,βj,uj,Δj)\n\n"

## Calling to PseudoArcLengthContinuation


### Initial value

In [7]:
function f_Example1!(F,u,p)

   x=u[1]
   y=u[2]
   z=u[3]
   F[1]=x^2+y^2+z^2-6
   F[2]=x+y-z
    
   return nothing 
end

f_Example1! (generic function with 1 method)

In [8]:
f=[0., 0.]
x0=[1.,1.,2.]  # kurban dago
p=nothing
#x0=[1.5,1.,2.] # Ez dago kurban
f_Example1!(f,x0,p)
f

2-element Vector{Float64}:
 0.0
 0.0

In [9]:
Beta0=zero(x0)
# Beta0[end]=1  singular
Beta0[1]=1
p=nothing
dt=0.1

0.1

In [11]:
#sol=PseudoArcLength(F2_Example1!, J2_Example1!, x0, β0, p)
#@show (sol.t, sol.x);

In [12]:
#f_Example1!(f,sol.x)
#f

### Step by step

#### Calcular Beta0

In [11]:
N=length(x0)
J2=zeros(N,N)

Betaj=copy(Beta0)
xj=copy(x0)

J2_Example1!(J2,x0,p,Beta0,x0,0.)
# compute beta
b=zero(x0)
b[end]=1
Betaj=J2\b
@show dot(Beta0,Betaj)
Betaj=Betaj/norm(Betaj);

dot(Beta0, Betaj) = 1.0


In [13]:
(dot(Beta0,Betaj), dot(Betaj,Betaj))

(0.7071067811865475, 0.9999999999999998)

In [17]:
N=length(x0)
F2=similar(x0)
J2=rand(N,N)

Betaj=copy(Beta0)
xj=copy(x0)
maxiters=100

dt=zero(eltype(x0))
F2_Example1!(F2,x0,p, Beta0,x0,dt)
if norm(F2)>0 
      Dx=zero(x0)
      Dx[1]=100
      iters=0
      while (norm(Dx)>1e-14 && iters<maxiters)
            println("iter=", iters)
            iters=iters+1 
            J2_Example1!(J2,xj,p,Betaj,x0,dt)
            lu_fact=lu(J2)
            F2_Example1!(F2,xj,p,Beta0,x0,dt)
            b=-F2
            Dx=lu_fact\b
            xj=xj+Dx
      end
      println(iters, "xj=", xj)
end

In [20]:
f_Example1!(f,xj,p)
f

2-element Vector{Float64}:
 0.0
 0.0